In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from utils.data_utils import clf_get_performance, get_and_assign_country, adjust_to_train_df, one_hot_encodings, FeaturePrep

# Note: Run data_analysis.ipynb first. Helper functions are in the utils folder, data is in the data folder

# Part 1) build an MVP and explore different models

In [2]:
df_fraud = pd.read_csv('data/Candidate_tech_evaluation_candidate_copy_data science_fraud.csv', index_col=0)
df_fraud['ip_address'] = df_fraud['ip_address'].astype(int)
df_ip_to_c = pd.read_excel('data/Candidate_tech_evaluation_candidate_copy_datascience_IpAddress_to_Country.xlsx')
df_fraud, df_ip = get_and_assign_country(df_fraud, df_ip_to_c)

In [3]:
#ad hoc feature engineering (later on I provide a more modularized approach)

# limit to frequent countries
frequent_countries = df_fraud[['country','user_id']].groupby('country').count().reset_index().sort_values('user_id', ascending = False).country[:30].values
df_fraud['country'] = [val if val in frequent_countries else 'Other' for val in df_fraud.country]

# One hot encoding of country, source, sex
ohe_columns= ['browser','sex','source','country'] #one hot encode text based fields
df_fraud = one_hot_encodings(df_fraud, ohe_columns, drop_first=True)

#add instant_purchase flag (see data analysis)
df_fraud['purchase_time'] = pd.to_datetime(df_fraud['purchase_time']) 
df_fraud['signup_time'] = pd.to_datetime(df_fraud['signup_time'])
#df_fraud['seconds_to_purchase'] = (df_fraud['purchase_time'] - df_fraud['signup_time']).dt.seconds 
df_fraud['instant_purchase'] = ((df_fraud['purchase_time'] - df_fraud['signup_time']).dt.seconds < 10)*1

#I add the month of signup and purchase as a feature in order to catch some possibility for seasonality
df_fraud['month_signup'] = [val.month for val in df_fraud.signup_time]
df_fraud['month_purchase'] = [val.month for val in df_fraud.purchase_time]
#do not use timestamps as additional features (see data exploration). drop device_id
df_fraud = df_fraud.drop(columns = ['signup_time', 'purchase_time', 'device_id'])
df_fraud = df_fraud.set_index('user_id')

In [4]:
#this shows that features beside instant_purchase are almost uncorrelated with the target, which will make modelling in general rather challenging
df_fraud.drop(columns = ['ip_address']).corr()['class'].abs().sort_values(ascending = False)

class                                   1.000000
instant_purchase                        0.704426
month_purchase                          0.309570
month_signup                            0.230131
source_Direct                           0.020302
browser_IE                              0.015082
country_Denmark                         0.013324
country_Germany                         0.011238
source_SEO                              0.010597
country_Canada                          0.010314
country_Poland                          0.009841
country_Sweden                          0.009570
country_Mexico                          0.008761
country_China                           0.008222
country_United_States                   0.008014
country_Romania                         0.007767
country_Norway                          0.007668
country_Spain                           0.006861
country_India                           0.006808
country_Netherlands                     0.006667
country_Viet_Nam    

In [5]:
#separate target and feature and perform train_test_split
y = df_fraud['class']
X = df_fraud.drop(columns = ['class'])
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)


In [6]:
#store all fraud ip addresses in the training set
drop_cols = ['ip_address']
fraud_ip_address = np.unique(X_train[y_train==1].ip_address)
ip_test = X_test.ip_address
X_train = X_train.drop(columns = drop_cols)
X_test = X_test.drop(columns = drop_cols)

#used to estimate costs later on
test_purchase = X_test.purchase_value


In [7]:
#use Logistic regression as MVP
clf_lr = LogisticRegression(max_iter = 1000)
clf_lr.fit(X_train,y_train)
y_pred = clf_lr.predict(X_test)
clf_get_performance(y_test,y_pred, test_purchase)

{'f1': 0.682,
 'recall': 0.521,
 'precision': 0.99,
 'accuracy': 0.954,
 'roc_auc': 0.76,
 'tn': 32571,
 'tp': 1776,
 'fn': 1635,
 'fp': 18,
 'tnr': 0.905,
 'tpr': 0.049,
 'fnr': 0.045,
 'fpr': 0.0,
 'pr_data': 0.095,
 'nr_data': 0.905,
 'Electronica_cost': 60733}

In [8]:
#add prediction of class=1 if ip_address had been fraudulent before
y_pred_adjust = np.array([1 if val in fraud_ip_address else y_pred[idx] for idx, val in enumerate(ip_test)])
clf_get_performance(y_test,y_pred_adjust, test_purchase)

{'f1': 0.672,
 'recall': 0.533,
 'precision': 0.91,
 'accuracy': 0.951,
 'roc_auc': 0.764,
 'tn': 32410,
 'tp': 1817,
 'fn': 1594,
 'fp': 179,
 'tnr': 0.9,
 'tpr': 0.05,
 'fnr': 0.044,
 'fpr': 0.005,
 'pr_data': 0.095,
 'nr_data': 0.905,
 'Electronica_cost': 60614}

In [9]:
#now try a LightGBM model with the default settings instead of a Logistic Regression -> slightly improves the results (cost)
#it seems that the feature on instant_purchase as well as the ip_address mechanism are the sole drivers of performance.
#I tried different versions of the LightGBM model and usually hyperparameter tuning it via GridSearchCV and minimize the metric (Electronica_cost for the evaluation set)
#is the way to go. However, due to the strong feature contribution imbalance, this did not give significantly different results.
#I also try to to adjust the thresholds for a positive score below by using probabilities directly but can stick with the 50%, which is working best

#clf_lgbm = LGBMClassifier(alpha = 1, n_estimators=200, num_leaves=50, reg_lambda=0.5)
clf_lgbm = LGBMClassifier()
clf_lgbm.fit(X_train,y_train)
y_pred = clf_lgbm.predict(X_test)
for v in range(20):
    y_pred = (clf_lgbm.predict_proba(X_test)[:,1]>v/20)*1
    y_pred_adjust = np.array([1 if val in fraud_ip_address else y_pred[idx] for idx, val in enumerate(ip_test)])
    p=clf_get_performance(y_test,y_pred_adjust, test_purchase)
    print(v/20,p['Electronica_cost'])

[LightGBM] [Info] Number of positive: 7854, number of negative: 76146
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001130 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 262
[LightGBM] [Info] Number of data points in the train set: 84000, number of used features: 41
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.093500 -> initscore=-2.271630
[LightGBM] [Info] Start training from score -2.271630
0.0 260712
0.05 104242
0.1 62517
0.15 60857
0.2 60534
0.25 60510
0.3 60518
0.35 60510
0.4 60510
0.45 60510
0.5 60510
0.55 60510
0.6 60510
0.65 60510
0.7 60510
0.75 60510
0.8 60510
0.85 60510
0.9 60510
0.95 60510


In [10]:
#As a third method, I try a clustering approach, where I cluster the train set and then assign probabilies based on the number of positive labels
#in the train set for each cluster. Next, I apply the clustering to the test set and then assign the computed probability to the test set data points 

scaler = StandardScaler()
#scae data for clusering
X_train_scaled= pd.DataFrame(data = scaler.fit_transform(X_train), columns = X_train.columns)
X_test_scaled = pd.DataFrame(data = scaler.transform(X_test), columns = X_test.columns)


In [11]:
#I first reduce with PCA and then perform the clustering
#The performance is better than the MVP but slightly worse than LightGBM


n_components = 10  # Adjust the number of components as needed
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X_train_pca)
cluster_train = kmeans.predict(X_train_pca)

cluster_dict = {}
for c in range(n_clusters):
    idx = np.where(cluster_train == c)[0]
    cluster_dict[c] = (np.mean(y_train.iloc[idx])>0.5)*1
cluster_dict

#Apply clustering to the reduced test set
test_cluster_labels = kmeans.predict(X_test_pca)
y_pred = np.array([cluster_dict[val] for val in test_cluster_labels])
y_pred_adjust = np.array([1 if val in fraud_ip_address else y_pred[idx] for idx, val in enumerate(ip_test)])
clf_get_performance(y_test,y_pred_adjust, test_purchase)


/Users/marcomenner/myenv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


{'f1': 0.669,
 'recall': 0.533,
 'precision': 0.897,
 'accuracy': 0.95,
 'roc_auc': 0.763,
 'tn': 32381,
 'tp': 1819,
 'fn': 1592,
 'fp': 208,
 'tnr': 0.899,
 'tpr': 0.051,
 'fnr': 0.044,
 'fpr': 0.006,
 'pr_data': 0.095,
 'nr_data': 0.905,
 'Electronica_cost': 60777}

# Part 2) Final Model 

In [12]:
#load data from scratch from scratch again 
df_fraud = pd.read_csv('data/Candidate_tech_evaluation_candidate_copy_data science_fraud.csv', index_col=0)
df_fraud['ip_address'] = df_fraud['ip_address'].astype(int)
df_ip_to_c = pd.read_excel('data/Candidate_tech_evaluation_candidate_copy_datascience_IpAddress_to_Country.xlsx')

In [13]:
df_test = df_fraud.iloc[119900:] #this can be replaced with the actual test data 

In [14]:
train_data = FeaturePrep(df_fraud,df_ip_to_c, ohe_columns, is_scoring=False)
train_data.get_features()
clf_final = LGBMClassifier()
clf_final.fit(train_data.X, train_data.y)

[LightGBM] [Info] Number of positive: 11265, number of negative: 108735
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001203 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 272
[LightGBM] [Info] Number of data points in the train set: 120000, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.093875 -> initscore=-2.267213
[LightGBM] [Info] Start training from score -2.267213


LGBMClassifier()

In [15]:
scoring_data = FeaturePrep(df_test,df_ip_to_c, ohe_columns, train_columns = train_data.X.columns, frequent_countries = train_data.frequent_countries, is_scoring=True)
scoring_data.get_features()


In [16]:
y_pred = clf_final.predict(scoring_data.X)
y_test = scoring_data.y
y_pred_adjust = np.array([1 if val in train_data.fraud_ip_address else y_pred[idx] for idx, val in enumerate(scoring_data.df_fraud.ip_address)])

clf_get_performance(y_test, y_pred_adjust, scoring_data.X.purchase_value)

{'f1': 1.0,
 'recall': 1.0,
 'precision': 1.0,
 'accuracy': 1.0,
 'roc_auc': 1.0,
 'tn': 89,
 'tp': 11,
 'fn': 0,
 'fp': 0,
 'tnr': 0.89,
 'tpr': 0.11,
 'fnr': 0.0,
 'fpr': 0.0,
 'pr_data': 0.11,
 'nr_data': 0.89,
 'Electronica_cost': 0}